In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
xml = """
<rule id="BRIEFCATCH_72217380358443072298334619098248039878" name="BRIEFCATCH_PUNCHINESS_921">
    <pattern>
        <token inflected="yes">call</token>
        <token>upon</token>
    </pattern>
    <message>Would shorter words add punch?|**Example** from Justice Gorsuch: “When **called on** to interpret a statute, this Court generally seeks to discern and apply the ordinary meaning of its terms at the time of their adoption.”|**Example** from Deanne Maynard: “The [order] merely confirms that it was not until later proceedings that he was **called on** to single out these waters.”</message>
    <suggestion><match no="1"/> on</suggestion>
    <suggestion><match no="1" postag="(V.*)" postag_regexp="yes" postag_replace="$1">ask</match></suggestion>
    <short>{"ruleGroup":null,"ruleGroupIdx":0,"isConsistency":false,"isStyle":true,"correctionCount":2,"priority":"2.84","WORD":true,"OUTLOOK":true}</short>
    <example correction="called on|asked">She was <marker>called upon</marker> three times.</example>
</rule>
"""

In [3]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
from utils.example_tag_validation import replace_examples, generate_corrected_examples

/Users/consult/miniconda3/envs/briefcatch/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
examples_list, usages = generate_corrected_examples(xml)

2024-02-16 13:30:45,764 [INFO] resp='{\n  "thought": "The provided rule has a pattern that matches the phrase \'call upon\' with suggestions to replace it with \'call on\' or \'ask\'. The example given correctly demonstrates the pattern with \'called upon\' being marked and provides appropriate corrections (\'called on\' or \'asked\'). Therefore, the example tag is correct and does not need to be changed or rewritten. There are no antipatterns provided in this rule, so there\'s no need to add or check for antipattern examples.",\n  "examples": [\n    "<example correction=\\"called on|asked\\">She was <marker>called upon</marker> three times.</example>"\n  ]\n}'
